In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from datetime import datetime

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)



# Split the Data into Training and Testing

In [6]:
y = df["loan_status"]

#loan status is the target
#other columns are 1 value only (static)
X = df[columns].drop(columns=["loan_status","pymnt_plan","hardship_flag","debt_settlement_flag"])
X=pd.get_dummies(X,columns=["home_ownership","verification_status","application_type","initial_list_status"])

#convert date to serial number (Excel style!)
def excel_date(date1):
    delta = date1 - datetime(1899, 12, 30)
    return float(delta.days) + (float(delta.seconds) / 86400)

X.issue_d = pd.to_datetime(X.issue_d).apply(excel_date)
X.next_pymnt_d = pd.to_datetime(X.next_pymnt_d).apply(excel_date)

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,43487.979627,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.526309,0.106747,0.357659,0.478007,0.373992,0.148001,0.860340,0.139660,0.123879,0.876121
std,10277.348590,0.048130,288.062432,1.155800e+05,22.160078,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.499311,0.308793,0.479314,0.499520,0.483865,0.355104,0.346637,0.346637,0.329446,0.329446
min,1000.000000,0.060000,30.890000,4.000000e+01,43466.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,43466.000000,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,43497.000000,19.760000,0.000000,0.000000,11.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,43497.000000,26.660000,0.000000,1.000000,16.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,43525.000000,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 89)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=128, random_state=1) 
rf_model = rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

In [11]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.8105734637062821

In [12]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,63,24
Actual 1,1763,15355


In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.72      0.90      0.07      0.81      0.64        87
   low_risk       1.00      0.90      0.72      0.95      0.81      0.66     17118

avg / total       0.99      0.90      0.73      0.94      0.81      0.66     17205



In [14]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07521112411413926, 'total_rec_int'),
 (0.0722907866674469, 'total_rec_prncp'),
 (0.060097615329767616, 'last_pymnt_amnt'),
 (0.0587262967292583, 'total_pymnt'),
 (0.051531596777888544, 'total_pymnt_inv'),
 (0.029260691277846133, 'issue_d'),
 (0.02732810723570911, 'int_rate'),
 (0.017878660068381486, 'dti'),
 (0.017637098706406326, 'out_prncp_inv'),
 (0.01679457795196158, 'annual_inc'),
 (0.01656289094566221, 'mths_since_recent_inq'),
 (0.016459279901802553, 'out_prncp'),
 (0.016425527301209508, 'il_util'),
 (0.015419512683711499, 'bc_util'),
 (0.01511677150789552, 'max_bal_bc'),
 (0.01483938225042884, 'installment'),
 (0.01474805757343753, 'mo_sin_old_rev_tl_op'),
 (0.014424241171055468, 'bc_open_to_buy'),
 (0.014055796552274505, 'all_util'),
 (0.014051374390339143, 'total_bal_ex_mort'),
 (0.013885252697870871, 'mths_since_rcnt_il'),
 (0.013809034282954522, 'total_bc_limit'),
 (0.013723010475464738, 'avg_cur_bal'),
 (0.013595548757599713, 'total_il_high_credit_limit'),
 (0.01324682

### Easy Ensemble AdaBoost Classifier

In [29]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
#eec = EasyEnsembleClassifier(random_state=1)
#eec.fit(X_train, y_train) 

rf_model = EasyEnsembleClassifier(random_state=1) 
rf_model = rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

In [30]:
# Calculated the balanced accuracy score

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.9197970678173006

In [31]:
# Display the confusion matrix

cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,78,9
Actual 1,975,16143


In [32]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.90      0.94      0.14      0.92      0.84        87
   low_risk       1.00      0.94      0.90      0.97      0.92      0.85     17118

avg / total       0.99      0.94      0.90      0.97      0.92      0.85     17205

